## Data Set and Optimization Solver

In [1]:
import pyepo

Auto-Sklearn cannot be imported.


In [2]:
# generate data
grid = (2,2) # grid size
num_data = 1000 # number of training data
num_feat = 5 # size of feature
deg = 4 # polynomial degree
e = 0.5 # noise width
feats, costs = pyepo.data.shortestpath.genData(num_data+1000, num_feat, grid, deg, e, seed=42)

In [3]:
from pyepo.model.grb import shortestPathModel
# set solver
optmodel = shortestPathModel(grid)
# test
optmodel.setObj(costs[0])
sol, obj = optmodel.solve()
print("Obj: {}".format(obj))
for i, e in enumerate(optmodel.arcs):
    if sol[i] > 1e-3:
        print(e)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-01
Obj: 0.5009689961416153
(0, 2)
(2, 3)


In [4]:
# split data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=1000, random_state=42)
# get training and test data set
dataset_train = pyepo.data.dataset.optDataset(optmodel, x_train, c_train)
dataset_test = pyepo.data.dataset.optDataset(optmodel, x_test, c_test)

Optimizing for optDataset...


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 4898.51it/s]


Optimizing for optDataset...


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 5047.34it/s]


In [5]:
# get data loader
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

## Prediction Model

In [6]:
import torch
from torch import nn

# build linear model
class LinearRegression(nn.Module):
    
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(num_feat, (grid[0]-1)*grid[1]+(grid[1]-1)*grid[0])
    
    def forward(self, x):
        out = self.linear(x)
        return out

In [7]:
# init model
reg = LinearRegression()

## Loss 

In [8]:
from torch.autograd import Function
import gurobipy as gp
from gurobipy import GRB

from pyepo import EPO

In [17]:
class polarConeAngle(nn.Module):
    """
    A autograd module for polar cone fitting loss with binary variables
    """
    
    def forward(self, pred_cost, true_sol):
        """
        Forward pass
        """
        loss = polarConeAngleFunc.apply(pred_cost, true_sol, optmodel)
        return loss

    
class polarConeAngleFunc(Function):
    """
    A autograd function for polar cone fitting loss with binary variables
    """
    
    @staticmethod
    def forward(ctx, pred_cost, true_sol, optmodel):
        # get device
        device = pred_cost.device
        # get batch size
        batch_size = len(pred_cost)
        # init loss
        loss = torch.empty(batch_size).to(device)
        for i in range(batch_size):
            # check if the vector in the cone
            if optmodel.modelSense == EPO.MINIMIZE:
                inside = torch.equal(pred_cost[i] <= 0, true_sol[i].type(torch.BoolTensor))
            if optmodel.modelSense == EPO.MAXIMIZE:
                inside = torch.equal(pred_cost[i] >= 0, true_sol[i].type(torch.BoolTensor))
            # zero loss if inside
            if inside:
                loss[i]= 0
            else:
                u = getProjection(pred_cost[i], true_sol[i], optmodel)
        print(loss)

        
def getProjection(cp, w, optmodel):
    """
    A function to get the projection of the vector onto the polar cone via solving a quadratic programming
    """
    # ceate a model
    m = gp.Model("shortest path")
    # turn off output
    m.Params.outputFlag = 0
    # bounds for constraints: vector on hyperquadrant
    lb, ub = []
    for wi in w:
        if optmodel.modelSense == EPO.MINIMIZE:
            if wi < 1e-5:
                pass
            else
                pass
        if optmodel.modelSense == EPO.MAXIMIZE:
            if wi < 1e-5:
                lb.append(-GRB.INFINITY)
                ub.append(0)
            else
                pass
    # varibles
    p = m.addVars(len(cp), name="x", lb=-GRB.INFINITY, ub=GRB.INFINITY)
    # onjective function
    obj = gp.quicksum((cp[i].item() - p[i]) ** 2 for i in range(len(cp)))
    m.setObjective(obj, GRB.MINIMIZE)
    # constraints: vector on hyperquadrant
    for i in (w < 1e-5).nonzero()[:,0].tolist():
        print(i)
    # solve
    m.update()
    m.optimize()
    # get solutions
    proj = [p[k].x for k in range(len(cp))]
    print(cp)
    print(proj)

In [18]:
# init loss
pca_loss = polarConeAngle()

## Warning: Numerical Issue for Arcos

In [19]:
torch.acos(torch.tensor([0, 1, 1+1e8]))

tensor([1.5708, 0.0000,    nan])

## Test 

In [20]:
for data in loader_train:
    x, c, w, z = data
    # forward pass
    cp = reg(x)
    loss = pca_loss(cp, w)
    break

1
3
tensor([ 0.0278,  1.1744, -1.6073, -0.5694], requires_grad=True)
[0.027828067541122437, 1.1744039058685303, -1.607252597808838, -0.5694019794464111]
0
2
tensor([ 0.0379,  0.5856, -0.5676,  0.1112], requires_grad=True)
[0.03790569305419922, 0.5855547785758972, -0.5676383972167969, 0.11117341369390488]
0
2
tensor([-0.6318,  0.9266, -0.9418, -0.5327], requires_grad=True)
[-0.6318060159683228, 0.9266294836997986, -0.9417837262153625, -0.532746434211731]
1
3
tensor([-0.7686,  0.0114,  0.4463,  0.0773], requires_grad=True)
[-0.7685568332672119, 0.011368781328201294, 0.4462822675704956, 0.07725083827972412]
0
2
tensor([0.1785, 0.1864, 0.4608, 0.4165], requires_grad=True)
[0.17852172255516052, 0.18637701869010925, 0.4608280658721924, 0.4164833128452301]
1
3
tensor([-0.8415,  0.4828, -0.7292, -0.2009], requires_grad=True)
[-0.8414782285690308, 0.482845664024353, -0.7291617393493652, -0.20090869069099426]
0
2
tensor([ 0.6891,  1.1526, -0.3066, -0.6096], requires_grad=True)
[0.689143419265747